# Importing Librabries and Data

In [5]:
import pandas as pd
import tensorflow as tf
import numpy as np
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Importing the dataset
df = pd.read_csv('Data/Churn_Modelling_Original.csv')

#Dropping Useless Columns
df.drop(columns=['RowNumber','CustomerId','Surname'],inplace=True)
df.head(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


# Feature Engineering

### 1. Categorical Variable Encoding
### 2. Scaling

## Categorical Variable Encoding

In [6]:
#Seperating Features and Labels
X = df.iloc[:,:-1].values
Y = df.iloc[:,-1].values

#Labeling The Binary variable Gender
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

#OneHotEncoding the remaining variable 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('enc', OneHotEncoder(), [1])], remainder='passthrough')
X = ct.fit_transform(X)[:,1:] #Transforming and dropping the first column to recover from dummy trap

pd.DataFrame(X).head(5)#Looking at the Coded Features

,0,1,2,3,4,5,6,7,8,9,10
0,0,0,619,0,42,2,0,1,1,1,101349
1,0,1,608,0,41,1,83807.9,1,0,1,112543
2,0,0,502,0,42,8,159661,3,1,0,113932
3,0,0,699,0,39,1,0,2,0,0,93826.6
4,0,1,850,0,43,2,125511,1,1,1,79084.1


Mapping:

Since 'Gender' was a binary variable it was encoded with the help of the 'LabelEncoder':

Female ----> 0   
  
Male ----> 1

The variable 'Geography' had more than 2 categories it was encoded using OneHotEncoding and subsequently one of the column was droped to recover from the dummy trap:

France ---> [0,0]   
  
Spain ----> [0,1]   
  
Germany ----> [1,0]

In [8]:
#Splitting
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=0)

#Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

#Saving a copy of the Scaled data
pd.DataFrame(x_train).to_csv('Data/x_train.csv',index=False)
pd.DataFrame(y_train).to_csv('Data/y_train.csv',index=False)
pd.DataFrame(x_test).to_csv('Data/x_test.csv',index=False)
pd.DataFrame(y_test).to_csv('Data/y_test.csv',index=False)

#Final Scaled Data
pd.DataFrame(x_train).head(5)

,0,1,2,3,4,5,6,7,8,9,10
0,-0.569844,1.743090,0.169582,-1.091687,-0.464608,0.006661,-1.215717,0.809503,0.642595,-1.032270,1.106432
1,1.754865,-0.573694,-2.304559,0.916013,0.301026,-1.377440,-0.006312,-0.921591,0.642595,0.968738,-0.748664
2,-0.569844,-0.573694,-1.191196,-1.091687,-0.943129,-1.031415,0.579935,-0.921591,0.642595,-1.032270,1.485335
3,-0.569844,1.743090,0.035566,0.916013,0.109617,0.006661,0.473128,-0.921591,0.642595,-1.032270,1.276528
4,-0.569844,1.743090,2.056114,-1.091687,1.736588,1.044737,0.810193,0.809503,0.642595,0.968738,0.558378


### About Standardization

Standardizing a dataset involves rescaling the distribution of values so that the mean of observed values is 0 and the standard deviation is 1. This can be thought of as subtracting the mean value or centering the data.

Standardization assumes that your observations fit a Gaussian distribution (bell curve) with a well-behaved mean and standard deviation

A value is standardized as follows:

![Standardization](_images/Standardization.png)

  
Where the mean is calculated as:

![Mean](_images/Mean.png)   

And the standard_deviation is calculated as:

![Standard Deviation](_images/Standard_Deviation.png)

# Model Building: Artificial Neural Network

While building the ANN, i will the sklearn function 'GridSearchCV' for tuning the hyperparameter of the ANN but since the ANN is made through the library keras with tensorflow backend the model would not be compatible with sklearn.  
To resolve this problem we have to use a sklearn wrapper given in the kears library that will take the keras ANN object and gives out a Classifier object that is compatible with the sklearn library and then we can encorporate the GridSearchCV function

To Use the keras wrapper for sklearn 'KerasClassifier'

We have to create the architecture of the ANN inside a builder function which will be passed inside the wrapper to generate the sklearn compatible ANN classifier

## Function to Build ANN Architecture

In [5]:
def build_classifier(optimizer):
    classifier = Sequential()
    #First Hidden Layer
    classifier.add(Dense(units=15, kernel_initializer='uniform', activation='relu', input_dim=11))
    classifier.add(Dropout(0.1))
    #Second Hidden Layer
    classifier.add(Dense(units=25, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dropout(0.1))
    #Third Hidden Layer
    classifier.add(Dense(units=15, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dropout(0.1))
    #Output Layer
    classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
    classifier.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

## HyperParameter Tuning Using GridSearchCV

In [6]:
classifier = KerasClassifier(build_fn=build_classifier)# Using the kears wrapper for sklearn 'KerasClassifier'

parameters = {'batch_size': [32, 64],
              'epochs': [100, 200],
              'optimizer': ['adam','rmsprop']} #Parameter space for GridSearch

grid_search = GridSearchCV(estimator=classifier,
                           param_grid=parameters,
                           scoring='accuracy',
                           cv=10)

grid_search = grid_search.fit(x_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

print("\nBest parameters:", best_parameters)
print("Best accuracy:", best_accuracy)

Streaming output truncated to the last 5000 lines.
113/113 [==============================] - 0s 3ms/step - loss: 0.3911 - accuracy: 0.8361
Epoch 103/200
113/113 [==============================] - 0s 3ms/step - loss: 0.3906 - accuracy: 0.8386
Epoch 104/200
113/113 [==============================] - 0s 3ms/step - loss: 0.3924 - accuracy: 0.8390
Epoch 105/200
113/113 [==============================] - 0s 3ms/step - loss: 0.3920 - accuracy: 0.8386
Epoch 106/200
113/113 [==============================] - 0s 3ms/step - loss: 0.3905 - accuracy: 0.8381
Epoch 107/200
113/113 [==============================] - 0s 3ms/step - loss: 0.3889 - accuracy: 0.8403
Epoch 108/200
113/113 [==============================] - 0s 3ms/step - loss: 0.3891 - accuracy: 0.8410
Epoch 109/200
113/113 [==============================] - 0s 3ms/step - loss: 0.3906 - accuracy: 0.8394
Epoch 110/200
113/113 [==============================] - 0s 4ms/step - loss: 0.3914 - accuracy: 0.8379
Epoch 111/200
113/113 [=============

### Note: The GridSearch results showcases that inside the parmeter space given the best accuracy reached on the training data is 85% which is good enough for segmentation problem and the hyperparameter for the best ANN configuration are:

### optimizer: 'adam',  batch_size: 32,  epochs: 200